# __Capstone Project - The Battle of Neighborhoods : Trip Advisor – Explore New Places__

### __Introduction__

XYZ is a travel agency company. The Company would like to predict/advice their customers who would like to explore new places the most efficient way they could plan their trip.
There are many places in the World which people would like to explore. One of the most important factors that needs to be considered while planning to explore a new city/place is time. We would be happier or satisfied in our trip, if we could explore more places within the available time. 
Therefore, it is advantageous for the company to predict the possible ways, by which their customers could make use of the time they are spending in the most efficient way. For example, the locations that would be covered in an area, what would be the weather condition, where do they need to plan the stay etc…   

Based on the area a customer would like to explore, with the help of location data of that area and the categories of interest of the customer, a suggestion needs to be provided to the customer.
This project aims to group the areas in a location as different clusters and make decisions based on it.

The XYZ Company would be able to provide helpful information for their customers and they would be able to explore the place accordingly. The customers would be highly satisfied with the service the Company is providing and this would help to increase the reputation of the company and gain more customers.

### __Data Description__

One city will be analysed in this project. Based on the city, the latitude and longitude values will be obtained by using geopy module.
The data used in this project is provided by Foursquare location data. The city’s latitude and longitude values will be used to extract the venues data from Foursquare location data using explore endpoint.
From the obtained results which is a JSON file, we will filter the columns and create a table/dataframe.

### __Import Libraries__

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

### __Define Foursquare Credentials__

In [2]:
CLIENT_ID = 'QX4VQKR04TYEAP5WBYVSZOMJUGFW3UXY4R45T3LU2VSFNROB' # your Foursquare ID
CLIENT_SECRET = 'VM4FEHITHAAG3ZQCJF1DUJ1R42JTRXSXENE1HPH0HUARG1BW' # your Foursquare Secret
VERSION = '20190817'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QX4VQKR04TYEAP5WBYVSZOMJUGFW3UXY4R45T3LU2VSFNROB
CLIENT_SECRET:VM4FEHITHAAG3ZQCJF1DUJ1R42JTRXSXENE1HPH0HUARG1BW


### __Define the city and get its latitude & longitude__

In [3]:
address = 'Udaipur, India'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

24.578721 73.6862571


### __Explore the Venues around the city__

In [4]:
# set the radius limit to be considered
radius = 150000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=QX4VQKR04TYEAP5WBYVSZOMJUGFW3UXY4R45T3LU2VSFNROB&client_secret=VM4FEHITHAAG3ZQCJF1DUJ1R42JTRXSXENE1HPH0HUARG1BW&ll=24.578721,73.6862571&v=20190817&radius=150000&limit=100'

In [5]:
# Send the GET Request and examine the results
results = requests.get(url).json()
print('There are {} Venues.'.format(len(results['response']['groups'][0]['items'])))
items = results['response']['groups'][0]['items']
items[0]

There are 56 Venues.


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4d70d94ccbc58cfa20167872',
  'name': 'Udaipur',
  'location': {'lat': 24.57850471985167,
   'lng': 73.68667602539061,
   'labeledLatLngs': [{'label': 'display',
     'lat': 24.57850471985167,
     'lng': 73.68667602539061}],
   'distance': 48,
   'postalCode': '313001',
   'cc': 'IN',
   'state': 'Rājasthān',
   'country': 'India',
   'formattedAddress': ['313001', 'Rājasthān', 'India']},
  'categories': [{'id': '50aa9e094b90af0d42d5de0d',
    'name': 'City',
    'pluralName': 'Cities',
    'shortName': 'City',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4d70d94ccbc58cfa20167872-0'}

In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

### __Display the obtained dataframe__

In [8]:
dataframe_filtered.head()

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,state,country,formattedAddress,address,city,crossStreet,id
0,Udaipur,City,24.578505,73.686676,"[{'label': 'display', 'lat': 24.57850471985167...",48,313001,IN,Rājasthān,India,"[313001, Rājasthān, India]",NaN,NaN,NaN,4d70d94ccbc58cfa20167872
1,Jagmandir Island Palace Hotel,Hotel,24.567959,73.678853,"[{'label': 'display', 'lat': 24.56795869952046...",1413,313001,IN,Rājasthān,India,"[Lake Pichola, Udaipur 313001, Rājasthān, India]",Lake Pichola,Udaipur,NaN,4bdf2cc77ea362b51e6043c4
2,The Oberoi Udaivilas,Resort,24.577199,73.672243,"[{'label': 'display', 'lat': 24.57719938258661...",1428,313001,IN,Rājasthān,India,"[Haridasji Ki Magri, Udaipur 313001, Rājasthān...",Haridasji Ki Magri,Udaipur,NaN,4ec25feb93ad36d7aa3289e2
3,Lalit Laxmi Vilas Palace Hotel,Hotel,24.593846,73.682412,"[{'label': 'display', 'lat': 24.59384554991198...",1728,NaN,IN,Rājasthān,India,"[Opposite Fateh Sagar Lake, Udaipur, Rājasthān...",Opposite Fateh Sagar Lake,Udaipur,NaN,4bc879a2dc55eee1557ce8ac
4,Fatehsagar Lake,Lake,24.599325,73.680491,"[{'label': 'display', 'lat': 24.59932510412237...",2366,313001,IN,Rājasthān,India,"[Nr. Moti Magri, Udaipur 313001, Rājasthān, In...",Nr. Moti Magri,Udaipur,NaN,4f674ebbe4b0cded5f00f0b2


### __Clean the Dataframe__

Identify the unique categories in the dataframe.

In [9]:
dataframe_filtered.categories.unique()

array(['City', 'Hotel', 'Resort', 'Lake', 'History Museum', 'Café',
       'Museum', 'Indian Restaurant', 'Restaurant', 'Historic Site',
       'Dessert Shop', 'Multiplex', 'Hotel Bar',
       'Mediterranean Restaurant', 'Rajasthani Restaurant', 'Roof Deck',
       'General Entertainment', 'Vegetarian / Vegan Restaurant',
       'Hookah Bar', 'Food', 'Castle', 'Monument / Landmark', 'Bistro',
       'Airport Lounge', 'Trail', 'Fast Food Restaurant', 'Coffee Shop'],
      dtype=object)

Delete the rows based on categories that are not of interest.

In [10]:
#delrow=('Tea Room','Hostel','Multiplex','Indie Movie Theater','Juice Bar', 'Snack Place')
delrow=('City','Multiplex')
dataframe_filtered1=dataframe_filtered
for i in delrow:
    dataframe_filtered1=dataframe_filtered1[dataframe_filtered1.categories != i]
dataframe_filtered1.head()

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,state,country,formattedAddress,address,city,crossStreet,id
1,Jagmandir Island Palace Hotel,Hotel,24.567959,73.678853,"[{'label': 'display', 'lat': 24.56795869952046...",1413,313001,IN,Rājasthān,India,"[Lake Pichola, Udaipur 313001, Rājasthān, India]",Lake Pichola,Udaipur,NaN,4bdf2cc77ea362b51e6043c4
2,The Oberoi Udaivilas,Resort,24.577199,73.672243,"[{'label': 'display', 'lat': 24.57719938258661...",1428,313001,IN,Rājasthān,India,"[Haridasji Ki Magri, Udaipur 313001, Rājasthān...",Haridasji Ki Magri,Udaipur,NaN,4ec25feb93ad36d7aa3289e2
3,Lalit Laxmi Vilas Palace Hotel,Hotel,24.593846,73.682412,"[{'label': 'display', 'lat': 24.59384554991198...",1728,NaN,IN,Rājasthān,India,"[Opposite Fateh Sagar Lake, Udaipur, Rājasthān...",Opposite Fateh Sagar Lake,Udaipur,NaN,4bc879a2dc55eee1557ce8ac
4,Fatehsagar Lake,Lake,24.599325,73.680491,"[{'label': 'display', 'lat': 24.59932510412237...",2366,313001,IN,Rājasthān,India,"[Nr. Moti Magri, Udaipur 313001, Rājasthān, In...",Nr. Moti Magri,Udaipur,NaN,4f674ebbe4b0cded5f00f0b2
5,Maharana Pratap Smarak,History Museum,24.598036,73.680831,"[{'label': 'display', 'lat': 24.59803588342685...",2219,313001,IN,Rājasthān,India,"[Moti Magari, Udaipur 313001, Rājasthān, India]",Moti Magari,Udaipur,NaN,4e84047761afba7222a4baeb


In [11]:
dataframe_filtered1.shape

(54, 15)

Delete the columns that are not required for this project.

In [12]:
#dataframe_filtered2=dataframe_filtered1.drop(['address','distance','labeledLatLngs','postalCode','cc','city','state','country','formattedAddress','crossStreet','neighborhood','id'], axis=1)
dataframe_filtered2=dataframe_filtered1.drop(['address','distance','labeledLatLngs','postalCode','cc','city','state','country','formattedAddress','crossStreet','id'], axis=1)
dataframe_filtered2.head()

,name,categories,lat,lng
1,Jagmandir Island Palace Hotel,Hotel,24.567959,73.678853
2,The Oberoi Udaivilas,Resort,24.577199,73.672243
3,Lalit Laxmi Vilas Palace Hotel,Hotel,24.593846,73.682412
4,Fatehsagar Lake,Lake,24.599325,73.680491
5,Maharana Pratap Smarak,History Museum,24.598036,73.680831


Reset the Index

In [13]:
dataframe_filtered3 = dataframe_filtered2.reset_index(drop=True)
dataframe_filtered3

,name,categories,lat,lng
0,Jagmandir Island Palace Hotel,Hotel,24.567959,73.678853
1,The Oberoi Udaivilas,Resort,24.577199,73.672243
2,Lalit Laxmi Vilas Palace Hotel,Hotel,24.593846,73.682412
3,Fatehsagar Lake,Lake,24.599325,73.680491
4,Maharana Pratap Smarak,History Museum,24.598036,73.680831
5,Radisson Blu,Resort,24.589976,73.667807
6,Brewmen Café,Café,24.601438,73.687107
7,Taj Lake Palace,Resort,24.575505,73.680170
8,City Palace Museum,Museum,24.576911,73.683544
9,Trident,Hotel,24.577066,73.669200


### __Generate map to visualize the positions of venues__

In [14]:
venues_map1 = folium.Map(location=[latitude, longitude], zoom_start=8) # generate map centred around City


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='City',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map1)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered3.lat, dataframe_filtered3.lng, dataframe_filtered3.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map1)

# display map
venues_map1

### __Apply k-means clustering to group the venues__

Create the dataframe with required columns to apply k-means clustering

In [15]:
dataframe_filtered4=dataframe_filtered3.drop(['name','categories'], axis=1)
dataframe_filtered4.head()

,lat,lng
0,24.567959,73.678853
1,24.577199,73.672243
2,24.593846,73.682412
3,24.599325,73.680491
4,24.598036,73.680831


Set the number of clusters to be considered and fit the dataframe

In [25]:
kclusters = 3
kmeans = KMeans(init="k-means++", n_clusters=3, n_init=12).fit(dataframe_filtered4)
#check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Display the centroid values

In [17]:
k_means_cluster_centers = kmeans.cluster_centers_
k_means_cluster_centers

array([[24.38204295, 72.56346962],
       [24.63910382, 73.68843379],
       [24.88556996, 74.63287765]])

Add clustering labels to the cleaned dataframe

In [18]:
#del dataframe_filtered5['Cluster labels']
dataframe_filtered5=dataframe_filtered3
#add clustering labels
dataframe_filtered5.insert(0,"Cluster labels",kmeans.labels_)
dataframe_filtered5

,Cluster labels,name,categories,lat,lng
0,1,Jagmandir Island Palace Hotel,Hotel,24.567959,73.678853
1,1,The Oberoi Udaivilas,Resort,24.577199,73.672243
2,1,Lalit Laxmi Vilas Palace Hotel,Hotel,24.593846,73.682412
3,1,Fatehsagar Lake,Lake,24.599325,73.680491
4,1,Maharana Pratap Smarak,History Museum,24.598036,73.680831
5,1,Radisson Blu,Resort,24.589976,73.667807
6,1,Brewmen Café,Café,24.601438,73.687107
7,1,Taj Lake Palace,Resort,24.575505,73.680170
8,1,City Palace Museum,Museum,24.576911,73.683544
9,1,Trident,Hotel,24.577066,73.669200


Create dataframes based on the cluster

In [19]:
dataframe_cluster0=dataframe_filtered5.loc[dataframe_filtered5['Cluster labels']==0]
dataframe_cluster1=dataframe_filtered5.loc[dataframe_filtered5['Cluster labels']==1]
dataframe_cluster2=dataframe_filtered5.loc[dataframe_filtered5['Cluster labels']==2]

In [20]:
#display shape each cluster
print('Cluster 0',dataframe_cluster0.shape)
print('Cluster 1',dataframe_cluster1.shape)
print('Cluster 2',dataframe_cluster2.shape)

Cluster 0 (2, 5)
Cluster 1 (50, 5)
Cluster 2 (2, 5)


Identify the unique categories in the Cluster dataframe.

In [21]:
# Check Different categories under the Cluster 0 venues
dataframe_cluster0.categories.unique()

array(['Indian Restaurant', 'Coffee Shop'], dtype=object)

In [22]:
# Check Different categories under the Cluster 1 venues
dataframe_cluster1.categories.unique()

array(['Hotel', 'Resort', 'Lake', 'History Museum', 'Café', 'Museum',
       'Indian Restaurant', 'Restaurant', 'Historic Site', 'Dessert Shop',
       'Hotel Bar', 'Mediterranean Restaurant', 'Rajasthani Restaurant',
       'Roof Deck', 'General Entertainment',
       'Vegetarian / Vegan Restaurant', 'Hookah Bar', 'Food', 'Castle',
       'Monument / Landmark', 'Bistro', 'Airport Lounge',
       'Fast Food Restaurant'], dtype=object)

In [23]:
# Check Different categories under the Cluster 2 venues
dataframe_cluster2.categories.unique()

array(['Trail', 'Castle'], dtype=object)

### __Generate map to visualize the Clusters__

Centroids of cluster is displayed as a black circle mark.
Spots in cluster 0 is displayed in blue.
Spots in cluster 1 is displayed in red.
Spots in cluster 2 is displayed in green.

In [24]:
venues_map00 = folium.Map(location=[latitude, longitude], zoom_start=14) # generate map centred around City

for lat, lng in (k_means_cluster_centers):
# add centroids of cluster as a black circle mark
    folium.CircleMarker(
    [lat, lng],
    radius=6,
    popup='Cluster_Center',
    fill=True,
    color='black',
    fill_color='black',
    fill_opacity=0.6
    ).add_to(venues_map00)


# add spots in cluster 0 to the map as blue circle markers
for lat, lng, name, categories in zip(dataframe_cluster0['lat'], dataframe_cluster0['lng'], 
                                           dataframe_cluster0['name'], dataframe_cluster0['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map00)

# add spots in cluster 1 to the map as red circle markers
for lat, lng, name, categories in zip(dataframe_cluster1['lat'], dataframe_cluster1['lng'], 
                                           dataframe_cluster1['name'], dataframe_cluster1['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map00)
    
# add spots in cluster 2 to the map as green circle markers
for lat, lng, name, categories in zip(dataframe_cluster2['lat'], dataframe_cluster2['lng'], 
                                           dataframe_cluster2['name'], dataframe_cluster2['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map00) 

# display map
venues_map00

From the map and dataframe it is clear that the maximum number of places can be explored in cluster0.
Cluster1 can be ignored as there are only two venues and they are Restaurant and Coffee Shop.
Cluster2 also has only venues but it is worth considering to explore the places in Cluster2 as there is a Trail and Castle.

Since we have obtained only two main venues in Cluster2 when the map was centered around Udaipur, let us explore Cluster2 to check if we could get more venues around it.
Using the latitude and longitude values of the centroid of Cluster2 explore the area and get the different venues around it.

In [83]:
# set the radius limit to be considered for centroid of cluster 2
radius2 = 2000

latitude2=k_means_cluster_centers[2][0]
longitude2=k_means_cluster_centers[2][1]
print(latitude2,longitude2)

24.88556996159449 74.6328776499459


In [84]:
# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, radius2, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=QX4VQKR04TYEAP5WBYVSZOMJUGFW3UXY4R45T3LU2VSFNROB&client_secret=VM4FEHITHAAG3ZQCJF1DUJ1R42JTRXSXENE1HPH0HUARG1BW&ll=24.88556996159449,74.6328776499459&v=20190817&radius=2000&limit=100'

In [85]:
# Send the GET Request and examine the results
results2 = requests.get(url).json()
print('There are {} Venues.'.format(len(results2['response']['groups'][0]['items'])))
items2 = results2['response']['groups'][0]['items']
items2[0]

There are 5 Venues.


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ceac0c0e1eeb60c8aeeafae',
  'name': 'Chittorgarh',
  'location': {'lat': 24.883557676983582,
   'lng': 74.62247839602007,
   'labeledLatLngs': [{'label': 'display',
     'lat': 24.883557676983582,
     'lng': 74.62247839602007}],
   'distance': 1073,
   'cc': 'IN',
   'city': 'Chittaurgarh',
   'state': 'Rājasthān',
   'country': 'India',
   'formattedAddress': ['Chittaurgarh', 'Rājasthān', 'India']},
  'categories': [{'id': '4bf58dd8d48988d159941735',
    'name': 'Trail',
    'pluralName': 'Trails',
    'shortName': 'Trail',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4ceac0c0e1eeb60c8aeeafae-0'}

In [88]:
dataframe2 = json_normalize(items2) # flatten JSON

# filter columns
filtered_columns02 = ['venue.name', 'venue.categories'] + [col for col in dataframe2.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered02 = dataframe2.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered02['venue.categories'] = dataframe_filtered02.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered02.columns = [col.split('.')[-1] for col in dataframe_filtered02.columns]

In [89]:
dataframe_filtered02.categories.unique()

array(['Trail', 'Castle', 'Train Station', 'History Museum'], dtype=object)

In [90]:
dataframe_filtered02

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,state,country,formattedAddress,address,city,crossStreet,id
0,Chittorgarh,Trail,24.883558,74.622478,"[{'label': 'display', 'lat': 24.88355767698358...",1073,NaN,IN,Rājasthān,India,"[Chittaurgarh, Rājasthān, India]",NaN,Chittaurgarh,NaN,4ceac0c0e1eeb60c8aeeafae
1,Chittorgarh Fort,Castle,24.887582,74.643277,"[{'label': 'display', 'lat': 24.88758224620540...",1073,NaN,IN,Rājasthān,India,"[Fort Road, Chitrakot, Rājasthān, India]",Fort Road,Chitrakot,NaN,5642dfed498ebfb8c529542d
2,Vijay Stambh,Castle,24.887778,74.646342,"[{'label': 'display', 'lat': 24.88777764298154...",1381,NaN,IN,Rājasthān,India,"[Chittorgarh Fort, Chittaurgarh, Rājasthān, In...",Chittorgarh Fort,Chittaurgarh,NaN,52c52d4511d2073357c93afa
3,Chittorgarh Railway Station,Train Station,24.875425,74.623451,"[{'label': 'display', 'lat': 24.87542529002411...",1477,NaN,IN,Rājasthān,India,"[Chittaurgarh, Rājasthān, India]",NaN,Chittaurgarh,NaN,4e29c90c1f6ebe5442412126
4,Victory Tower,History Museum,24.874298,74.623502,"[{'label': 'display', 'lat': 24.87429806175967...",1571,NaN,IN,Rājasthān,India,"[Chittorgarh Fort, Chittaurgarh, Rājasthān, In...",Chittorgarh Fort,Chittaurgarh,NaN,4dcd482f183899ddfab462e5


From the above observations of Cluster2, two extra spots (Castle and Museum) has been identified.
Also as per the venue list there are no Hotels around Cluster2. But there is a Train Station which is an useful information.

### __Conclusion__

In this study, I analyzed that to explore Udaipur it would be better to allocate more time to explore the places in Cluster0 and to prefer stay in one of the Hotels which is closer to the centroid of Cluster0 and then probably to explore the places in Cluster2. Rail Transport can be preferred to expore Cluster2 and the places in Cluster2 can be covered in a single day as the distance to different venues are within 3km. The western side of Udaipur can be given the least importance as there are not much interesting places to explore.